In [3]:
#Doing Array operations if needed
import numpy as np

#Making the NN
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

#Loading images from disk
from scipy import misc
#Getting the name of all the files
import glob
#Handling data in df and saving to csv
import pandas as pd

#One hot encoding of labels
from sklearn.preprocessing import LabelBinarizer
#Splitting the data set for train and validation if needed
#You can also use validation split
from sklearn.model_selection import train_test_split
#Handling pickle files
import pickle

Using TensorFlow backend.


In [3]:
#Load target labels
train_labels=pd.read_csv('dataset/trainLabels.csv')
train_labels.set_index('id', inplace=True)
train_labels=train_labels.to_dict(orient='id')

In [4]:
#Loading the dataset
x_train=[]
y_train=[]

count=0
for image_path in glob.glob("dataset/train/*.png"):
    count+=1
    if (count%5000==0):
        print(count)
    x_train.append(misc.imread(image_path))
    temp=image_path.split('\\')[-1].split('.')[0]
    label=train_labels[int(temp)]['label']
    y_train.append(label)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [ ]:
#For Debugging purposes
x_train=np.array(x_train)
print(x_train.shape)

In [5]:
lb = LabelBinarizer()
y_train=lb.fit_transform(y_train)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [7]:
# save the labelBinarizer to disk if you dont want to call fit everytime
from sklearn.externals import joblib
joblib.dump(lb, 'label.pkl')

['label.pkl']

In [27]:
#This can be a faster way to save/load as compared to loading images everytime
file = open('x_train', 'wb')
pickle.dump(x_train,file)
file.close()
file = open('y_train', 'wb')
pickle.dump(y_train,file)
file.close()

In [2]:
#This can be a faster way to save/load as compared to loading images everytime
with open('x_train', 'rb') as f:
    x_train = pickle.load(f) 

with open('y_train', 'rb') as f:
    y_train = pickle.load(f) 

In [ ]:
#Normalizin the images
x_train = x_train.astype('float32')
x_train/= 255

In [3]:
#Train and val split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [5]:
#This is used for augmentation of train dataset 
imdgen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False,  # randomly flip images
)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
imdgen.fit(x_train)

# fit the model on the batches generated by datagen.flow()
dgen = imdgen.flow(x_train, y_train, batch_size=128)


In [6]:
# For reproducibility
np.random.seed(1000)

# Create the model
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='elu',kernel_initializer='he_normal', input_shape=(32, 32, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='elu',kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3, 3), activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='elu',kernel_initializer='he_normal'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='elu',kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1024, activation='elu',kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
checkpoint = ModelCheckpoint("CIFAR_10.h5", monitor='val_loss', verbose=2, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 26, 26, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 128)       73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 9, 9, 128)         147584    
__________

In [7]:
#Load the best weights if you are going to train model from a previous checkpoint

#model.load_weights('CIFAR_10.h5')
# Train the model with modified images as well
#model.fit_generator(dgen,
#                    steps_per_epoch=x_train.shape[0]/128,
#                    epochs=500,
#                    validation_data=(x_val, y_val),
#                    workers=2,callbacks=callbacks_list)

# Train the model with provided images
model.fit(x_train, y_train,batch_size=128,epochs=500,
                    validation_data=(x_val, y_val),callbacks=callbacks_list,shuffle=True)

In [76]:
#making output file
#This probably takes a while
#I had to do it this way because of limited memory of my laptop 6 gigs :(
output=pd.DataFrame(columns=['id','label'])
count=0
for image_path in glob.glob("dataset/test/test/*.png"):
    count+=1
    if (count%5000==0):
        print(count)
    img=misc.imread(image_path).astype('float32')
    img/= 255
    img=np.array([img])
    pred=model.predict(img)
    pred=lb.inverse_transform(pred)
    index=image_path.split('\\')[-1].split('.')[0]
    output=output.append({'id':index, 'label':pred[0]}, ignore_index=True)


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000


In [79]:
#Write the output to a csv
output.to_csv('output_custom.csv',index=False)